In [ ]:
import numpy as np
import pandas as pd
import matplotlib,pyplot as plt
import os


import keras

from keras.preprocessing.image import  ImageDataGenerator, img_to_array ,load_img
from keras.applications.vgg19 import VGG19, preprocess_imput, decode_predictions

In [ ]:
train_datagen = ImageDataGenerator(zoom_range= 0.5, shear_range= 0.3, horizontal_flip= True, preprocessing_function= preprocess_imput )

val_datagen = ImageDataGenerator(preprocessing_function= preprocess_imput )

In [ ]:
train = train_datagen.flow_from_directory(directory= "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                          target_size= (256,256), batch_size=32)

val = val_datagen.flow_from_directory(directory= "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                        target_size= (256,256), batch_size=32)

In [ ]:
t_img , label = train.next()

In [ ]:
def plotImage(img_arr, label):
    
    for im , l in zip(img_arr , label):
        plt.figure(figsize=(5,5))
        plt.imshow(im/255)
        plt.show()
        

In [ ]:
plotImage(t_img[:3], label[:3])

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications,vgg19 import VGG19
import keras

In [ ]:
base_model = VGG19(input_shape=(256,256,3), include_top=False)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
X = Flatten()(base_model.output)

X = Dense(units= 38, activation='softmax')(x)


#Creating our model
model = Model(base_model.input, x)

In [ ]:
model.summary()

Model: "model"

In [ ]:
model.compile(optimizer= 'adam' , loss= keras.losses.catagorical_crossentrophy , metrics= ['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stopping
es = EarlyStopping(monitor= 'val_accuracy', min_delta= 0.01, patience= 3,verbose=1)

# model check point
mc = ModelCheckpoint(filepath="best_model.h5",
                     monitor= 'val_accuracy',
                     min_delta= 0.01,
                     patience= 3,
                     verbose=1 ,
                     save_best_only= True)


cb = [es, mc]

In [ ]:
his = model.fit_generator(train ,
                          steps_per_epoch= 16,
                          epochs= 50,
                          verbose= 1,
                          callbacks= cb ,
                          validations_data= val ,
                          validation_steps= 16 )

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] , c = "red")
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'] , c = "red")
plt.title("loss vs v-loss")
plt.show()

In [ ]:
# load best model

from keras.models import load_model

model = load_model("/content/best_model.h5")

In [ ]:
acc = model.evaluate_generator(val)[1]


print(f"The accuracy of your model is = {acc*100} %")

In [ ]:
ref = dict(zip( list(train.class_indices.values()) , list(train.class_indices.keys()) ) )

In [ ]:
def prediction(path):
    
    img = load_img(path, target_size= (256,256))
    
    i = img_ to_array(img)
    
    im = preprocess_input(i)
    
    img = np.expand_dims(im , axis= 0)
    
    pred = np.argmax(model.predict(img) )
    
    print(f" the image belongs to { ref[pred] } ")

In [ ]:
path = "/content/test/test/TomatoYellowCurlVirus4.JPG"

prediction(path)